# Task 4: General Health Query Chatbot (Prompt Engineering Based)

## Problem Statement
Users often ask general health questions online, but responses can be unsafe if a system gives
medical diagnosis or dosage instructions. We need a chatbot that can provide **general, safe,
friendly** health information while avoiding harmful advice.

## Goal
Build a chatbot that:
- Accepts user health questions
- Sends queries to an LLM through a **real API** (DeepSeek)
- Uses **prompt engineering** to produce clear, friendly responses
- Includes **safety filtering** to avoid dangerous or harmful medical advice
- Provides a simple conversational experience in a Jupyter Notebook

---


## Tools & Technology Used

### 1) LLM (Large Language Model)
We use an LLM to generate natural language answers for general health questions.

### 2) API Integration (DeepSeek API)
Instead of training our own model, we call an online LLM using an API request:
- Send: system prompt + conversation history + user message
- Receive: assistant response

### 3) Prompt Engineering
We control the assistant behavior using a **system prompt**, e.g.:
- "Act like a helpful medical assistant"
- "Do not diagnose"
- "Avoid medication dosage instructions"
- "Encourage doctor visit for serious symptoms"

### 4) Safety Filters
We implement safety in two ways:
- **Pre-safety filter**: blocks dangerous/self-harm related prompts before calling the API
- **Post-safety filter**: adds disclaimers and flags risky content (e.g., dosage patterns)

### 5) UI inside Jupyter
We use `ipywidgets` to build a simple chat interface in the notebook.

---


In [9]:
import os
import re
import requests
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML



## API Key Security (Best Practice)

For security, the DeepSeek API key is **NOT hard-coded** in this notebook.
It is loaded from a Windows Environment Variable:

- Variable name: `DEEPSEEK_API_KEY`

This prevents accidental key exposure when sharing the notebook or uploading to GitHub.

---


In [8]:
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY", "")
DEEPSEEK_URL = "https://api.deepseek.com/chat/completions"
DEEPSEEK_MODEL = "deepseek-chat"

if not DEEPSEEK_API_KEY:
    print("⚠️ DEEPSEEK_API_KEY not found. Set it in Environment Variables and restart Jupyter.")
else:
    print("✅ DeepSeek key loaded (hidden).")


✅ DeepSeek key loaded (hidden).


## Prompt Engineering: System Prompt

The system prompt defines the chatbot rules and style:

- Friendly tone
- General health info only
- No diagnosis
- No dosage or prescription instructions
- Suggest consulting a doctor for serious symptoms
- Add empathy if user sounds anxious/sad

---


In [10]:
SYSTEM_PROMPT = """
You are a friendly, empathetic, and cautious health information assistant.

Rules:
- Provide general health information only.
- Do NOT diagnose conditions.
- Do NOT give medication dosages or prescribing instructions.
- If symptoms are severe, worsening, or urgent, advise seeking medical care.
- Keep answers clear, structured, and beginner-friendly.
- If the user sounds anxious or sad, respond with warmth and reassurance.
"""


## Safety Handling

We apply two safety layers:

### 1) Pre-safety filter
Blocks dangerous self-harm / overdose content before the API is called.

### 2) Post-safety filter
Adds a disclaimer to every response and flags risky patterns (e.g., dosage text).

---


In [11]:
DANGEROUS_KEYWORDS = [
    "suicide", "kill myself", "end my life", "self-harm",
    "overdose", "poison", "die"
]

RISKY_PATTERNS = [
    r"\b\d+\s*(mg|milligram|tablet|pill|pills)\b",
    r"\btake\s+\d+\s*(mg|milligram|tablet|pill|pills)\b"
]

def pre_safety_filter(text: str):
    t = text.lower()
    for kw in DANGEROUS_KEYWORDS:
        if kw in t:
            return (
                "I'm really sorry you're feeling this way. 💛\n"
                "I can’t help with self-harm or overdose situations.\n\n"
                "Please seek immediate help from local emergency services or a trusted person."
            )
    return None

def post_safety_filter(reply: str) -> str:
    # Flag dosage-like patterns
    for pattern in RISKY_PATTERNS:
        if re.search(pattern, reply, flags=re.IGNORECASE):
            reply += "\n\n⚠️ Medication dosing must be decided by a licensed doctor."
            break

    disclaimer = (
        "\n\n⚠️ *This is general information, not medical advice. "
        "For diagnosis or treatment, consult a healthcare professional.*"
    )
    if disclaimer not in reply:
        reply += disclaimer

    return reply


## Advanced Feature: Emotion-Aware Responses

We detect a simple emotional tone in the user's message:
- anxious
- sad
- neutral

This emotion is attached to the user message so the LLM can reply more appropriately.

---


In [13]:
def detect_emotion(text: str) -> str:
    t = text.lower()
    if any(w in t for w in ["worried", "anxious", "panic", "scared", "stressed"]):
        return "anxious"
    if any(w in t for w in ["sad", "depressed", "hopeless"]):
        return "sad"
    return "neutral"


## DeepSeek API Call

This function sends:
- system prompt
- conversation history
- current user query

to DeepSeek and returns the model response.

---


In [14]:
def call_deepseek(messages):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": DEEPSEEK_MODEL,
        "messages": messages,
        "temperature": 0.4
    }

    try:
        r = requests.post(DEEPSEEK_URL, headers=headers, json=payload, timeout=15)
        r.raise_for_status()
        return r.json()["choices"][0]["message"]["content"].strip()
    except requests.exceptions.Timeout:
        return "API Error: Request timed out. Please try again."
    except Exception as e:
        return f"API Error: {e}"


## Chatbot Logic (Memory + Safety + Emotion)

Pipeline:
1. Pre-safety check (block dangerous)
2. Add system prompt
3. Add last few conversation turns (memory)
4. Add emotion tag to user message
5. Call DeepSeek API
6. Post-safety filter + disclaimer

---


In [15]:
def health_chatbot(user_msg: str, history_pairs):
    # 1) Pre-safety
    blocked = pre_safety_filter(user_msg)
    if blocked:
        return post_safety_filter(blocked)

    # 2) Build messages for DeepSeek
    msgs = [{"role": "system", "content": SYSTEM_PROMPT}]

    # Memory: include last 6 message pairs
    for u, b in history_pairs[-6:]:
        msgs.append({"role": "user", "content": u})
        msgs.append({"role": "assistant", "content": b})

    # Emotion tag
    emo = detect_emotion(user_msg)
    msgs.append({"role": "user", "content": f"[Emotion: {emo}] {user_msg}"})

    # 3) LLM call
    reply = call_deepseek(msgs)

    # 4) Post-safety
    return post_safety_filter(reply)


## Interactive Chat UI (Jupyter)

This provides:
- input box
- send button
- chat bubble display
- conversation memory

---


In [16]:
chat_history = []

chat_box = widgets.Output(layout=widgets.Layout(
    border="1px solid #ddd",
    padding="10px",
    height="420px",
    overflow_y="auto"
))

status = widgets.HTML(value="<i>Ready.</i>")
emotion_label = widgets.HTML(value="<b>Emotion:</b> -")

input_box = widgets.Textarea(
    placeholder="Type your health question here...\nThen click Send.",
    layout=widgets.Layout(width="100%", height="80px")
)

send_btn = widgets.Button(description="Send", button_style="primary")
clear_btn = widgets.Button(description="Clear Chat", button_style="warning")

def add_bubbles(user_text, bot_text):
    ts = datetime.datetime.now().strftime("%H:%M")
    user_html = f"""
    <div style="margin:10px 0;">
      <div style="background:#DCF8C6;padding:10px 12px;border-radius:14px;max-width:75%;display:inline-block;">
        <b>You</b> <span style="color:#666;font-size:12px;">{ts}</span><br>{user_text}
      </div>
    </div>
    """
    bot_html = f"""
    <div style="margin:10px 0;">
      <div style="background:#F1F0F0;padding:10px 12px;border-radius:14px;max-width:75%;display:inline-block;">
        <b>Bot</b><br>{bot_text}
      </div>
    </div>
    """
    with chat_box:
        display(HTML(user_html))
        display(HTML(bot_html))

def on_send(_):
    user_text = input_box.value.strip()
    if not user_text:
        return

    emo = detect_emotion(user_text)
    emotion_label.value = f"<b>Emotion:</b> {emo.capitalize()}"
    status.value = "<b>⏳ Thinking…</b>"

    input_box.disabled = True
    send_btn.disabled = True

    bot_text = health_chatbot(user_text, chat_history)
    chat_history.append((user_text, bot_text))
    add_bubbles(user_text, bot_text)

    input_box.value = ""
    input_box.disabled = False
    send_btn.disabled = False
    status.value = "<i>Ready.</i>"

send_btn.on_click(on_send)

def on_clear(_):
    chat_history.clear()
    chat_box.clear_output()
    emotion_label.value = "<b>Emotion:</b> -"
    status.value = "<i>Chat cleared. Ready.</i>"
    input_box.value = ""

clear_btn.on_click(on_clear)

title = widgets.HTML("""
<h2 style="margin-bottom:0;">🩺 Health Information Assistant</h2>
<p style="margin-top:4px;color:#555;">
General health questions • DeepSeek API • Safety filters • Emotion-aware responses
</p>
""")

controls = widgets.HBox([send_btn, clear_btn])
meta = widgets.HBox([emotion_label, status])

display(title, chat_box, meta, input_box, controls)

with chat_box:
    display(HTML("<i>Try: What causes a sore throat? • Is paracetamol safe for children?</i>"))


HTML(value='\n<h2 style="margin-bottom:0;">🩺 Health Information Assistant</h2>\n<p style="margin-top:4px;color…

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…

Textarea(value='', layout=Layout(height='80px', width='100%'), placeholder='Type your health question here...\…

## Example Queries & Observations

Try asking:
- "What causes a sore throat?"
- "Is paracetamol safe for children?"
- "How can I improve my heart health?"
- "I feel anxious about my health, what should I do?"

**Expected behavior:**
- Friendly, clear answers
- No diagnosis
- No dosage instructions
- Disclaimers included
- More empathetic tone for anxious/sad messages

---


## Final Insights

This project demonstrates how an LLM can be used safely for general health information using:
- Prompt engineering
- API-based LLM integration
- Safety filters
- Basic emotion-aware responses
- A simple conversational UI

## Limitations
- The chatbot is not a medical professional and cannot diagnose.
- Responses depend on the LLM and internet/API availability.
- The safety filters here are basic keyword/pattern checks.

## Future Improvements
- Add stronger medical safety classification (rule-based + ML)
- Add multilingual support (English/Urdu)
- Add citations from trusted sources (WHO/CDC/NHS)
- Add logging + analytics (emotion counts, response time)

---
